In [13]:
# For files e data manipulations
from imports_ import *
import os
import shutil
import random

## Neural Network

In [14]:
pnn = PreNeuralNetwork(train='C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/train', test='C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/test' ,
                    valid='C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/validation')

In [15]:
dir_train = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/train'
dir_test  = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/test'
dir_valid = 'C:/Users/Lucas/Documents/PIBIC/DATASET/NIH-CHEST/archive/validation'

batch_size = 4  
width = 224
length = 224
channel = 3
image_size = (width, length)
random.seed(123) 

Como está dando o CLOCK_WATCHDOG_TIMEOUT reduzirei a quantidade de imagens

In [17]:
train_datagen= ImageDataGenerator(rescale=1.0/255)
val_datagen  = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator=train_datagen.flow_from_directory(dir_train,
                                        target_size=image_size,
                                        color_mode='rgb',  # Ensure RGB color mode
                                        class_mode='categorical',  # Adjust class_mode as needed
                                        batch_size=batch_size,
                                        subset='training'  # Specify 'training' for the training set
                                    )

val_generator=val_datagen.flow_from_directory(dir_valid,
                                        target_size=image_size,
                                        color_mode='rgb',
                                        class_mode='categorical',
                                        batch_size=batch_size
                                    )

test_generator=test_datagen.flow_from_directory(dir_test,
                                        target_size=image_size,
                                        color_mode='rgb',
                                        class_mode='categorical',
                                        batch_size=batch_size
                                    )

Found 77877 images belonging to 9 classes.
Found 4328 images belonging to 9 classes.
Found 4326 images belonging to 9 classes.


In [18]:
n_classes = train_generator.num_classes
print("Number of classes:", n_classes)

Number of classes: 9


In [20]:

if tf.test.is_gpu_available():
    print("GPU is available and accessible.")
else:
    print("GPU is not available.")


GPU is not available.


In [21]:
# Assuming you have a GPU with index 0
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Specify GPU device
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


In [22]:
516berb

SyntaxError: invalid decimal literal (318676807.py, line 1)

In [ ]:
# Create an instance of ResNet101Model
num_classes = n_classes  # Adjust the number of classes as needed
image_shape = (width,length,channel)  # Adjust image dimensions as needed
model = ResNet101Model(num_classes, image_shape)

# Train the model
history = model.train(train_generator, val_generator, epochs=1, batch_size=batch_size, early_stop_patience=5)

Epoch 1/10
19470/19470 [==============================] - 4666s 240ms/step - loss: 1.1642 - accuracy: 0.6965 - val_loss: 1.1216 - val_accuracy: 0.7077
Epoch 2/10
19470/19470 [==============================] - 4491s 231ms/step - loss: 1.1547 - accuracy: 0.6967 - val_loss: 1.1328 - val_accuracy: 0.7077
Epoch 3/10
19470/19470 [==============================] - 4496s 231ms/step - loss: 1.1503 - accuracy: 0.6967 - val_loss: 1.1120 - val_accuracy: 0.7077
Epoch 4/10
19470/19470 [==============================] - 4596s 236ms/step - loss: 1.1475 - accuracy: 0.6967 - val_loss: 1.1111 - val_accuracy: 0.7077
Epoch 5/10
19470/19470 [==============================] - 4716s 242ms/step - loss: 1.1449 - accuracy: 0.6967 - val_loss: 1.1082 - val_accuracy: 0.7077
Epoch 6/10
19470/19470 [==============================] - 4678s 240ms/step - loss: 1.1431 - accuracy: 0.6967 - val_loss: 1.1067 - val_accuracy: 0.7077
Epoch 7/10
19470/19470 [==============================] - 4832s 248ms/step - loss: 1.1418 - ac

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)

1082/1082 [==============================] - 218s 202ms/step - loss: 1.1143 - accuracy: 0.7027


In [ ]:
full_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [ ]:
# # Checkpoint
# checkpoint = tf.keras.callbacks.ModelCheckpoint(f'model_NIG-CHEST/model_vgg_{qntd_for_model}_1.h5', 
#                                                 verbose = 1, 
#                                                 save_best = True, 
#                                                 save_weights_only = True)

# # Early stop
# early_stop = tf.keras.callbacks.EarlyStopping(patience = 4) 

In [ ]:
plot_history(history)